## Imports
Execute the cell below to import all packages needed.

In [1]:
# System functionality
import sys
sys.path.append('..')

import glob
import os
import numpy as np 
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import seaborn as sns
import graphviz
import math as math
import json
import numpy
import cv2
import scipy.stats as stats
import matplotlib.image as mpimg
import sklearn.feature_selection as feature_selection
from skimage import data, io, filters
from numpy import linspace
from numpy.core.umath import pi
from matplotlib.ticker import FuncFormatter
from matplotlib.font_manager import FontProperties
from emoji_extractor.extract import Extractor
from emojipedia import Emojipedia
from skimage import io
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import chi2_contingency
from scipy.interpolate import spline

## Load csv into pandas dataframe

In [2]:
path__tweets_csv = '/home/stelios/Desktop/Honours Project/Samples/tweet_data/tweets_lon_all.csv'
path_users_csv = '/home/stelios/Desktop/Honours Project/Samples/user_data/lon.csv'

lon_join_df = pd.read_csv(path__tweets_csv,index_col=None, header=0, low_memory = False).applymap(str).join(pd.read_csv(path_users_csv,index_col=None, header=0, low_memory = False).applymap(str).set_index('user_id'), on='user_id')

In [3]:
lon_join_df.columns

Index(['user_id', 'user_name', 'user_screen_name', 'user_statuses_count',
       'tweet_id', 'hashtags', 'is_quote_status', 'text', 'created_at',
       'source', 'in_reply_to_screen_name', '_unit_id', '_golden',
       '_unit_state', '_trusted_judgments', '_last_judgment_at', 'ethnicity',
       'ethnicity:confidence', 'gender', 'gender:confidence',
       'reason_not_valid', 'reason_not_valid:confidence', 'valid_photo',
       'valid_photo:confidence',
       'which_icon_most_closely_matches_the_persons_skin_tone',
       'which_icon_most_closely_matches_the_persons_skin_tone:confidence',
       'orig__golden', 'ethnicity_gold', 'ethnicity_gold_reason',
       'gender_gold', 'gender_gold_reason', 'image_url',
       'reason_not_valid_gold', 'reason_not_valid_gold_reason',
       'valid_photo_gold', 'valid_photo_gold_reason',
       'which_icon_most_closely_matches_the_persons_skin_tone_gold',
       'which_icon_most_closely_matches_the_persons_skin_tone_gold_reason'],
      dtype='ob

In [4]:
# Gia kathe user:
# 1) Number of tweets 2) Number of tweets with emojis

# For all users (1) for all tweets, (2) for only emoji tweets:
# 1) Min number of tweets , 2) Max number of tweets, 3) Average number of tweets

In [5]:
# Load users
lon_users_df = pd.read_csv(path_users_csv,index_col=None, header=0, low_memory = False).applymap(str)

In [6]:
# Fill pandas dataframe 
emojis_count_create_df = pd.DataFrame(columns = ['User','Tweets','Tweets_with_Emoji','Gender','Race'])
emojis_count_create_df['User'] = lon_users_df['user_id']
emojis_count_create_df['Gender']= lon_users_df['gender']
emojis_count_create_df['Race']= lon_users_df['ethnicity']
emojis_count_create_df

User Tweets Tweets_with_Emoji  Gender   Race
0               19243274    NaN               NaN  female  white
1              506251999    NaN               NaN     nan    nan
2             2744375046    NaN               NaN     nan    nan
3             2998532867    NaN               NaN  female  black
4             1789459591    NaN               NaN     nan    nan
5              321527153    NaN               NaN     nan    nan
6              295785833    NaN               NaN    male  white
7               55855192    NaN               NaN     nan    nan
8               60567230    NaN               NaN     nan    nan
9              116461862    NaN               NaN     nan    nan
10             285089147    NaN               NaN  female    nan
11             483379256    NaN               NaN  female    nan
12              41088091    NaN               NaN     nan    nan
13              46188134    NaN               NaN     nan    nan
14    806877440693960704    NaN               NaN     nan    nan
15              29734571    NaN               NaN     nan    nan
16             113596205    NaN               NaN     nan    nan
17    870383142829527041    NaN               NaN    male  white
18    934205270313656320    NaN               NaN     nan    nan
19            2857070033    NaN               NaN  female  white
20              19318400    NaN               NaN    male  white
21             221445059    NaN               NaN    male  white
22             465774620    NaN               NaN     nan    nan
23             173235703    NaN               NaN  female  white
24            1408212338    NaN               NaN    male  white
25             455253966    NaN               NaN     nan    nan
26              20132059    NaN               NaN    male  white
27              19297156    NaN               NaN    male  white
28            3071081479    NaN               NaN     nan    nan
29    899583323961524224    NaN               NaN    male  white
...                  ...    ...               ...     ...    ...
9970            96158885    NaN               NaN     nan    nan
9971            49093116    NaN               NaN     nan    nan
9972  721050612209750016    NaN               NaN     nan    nan
9973           459502210    NaN               NaN    male  black
9974          2765006192    NaN               NaN  female  white
9975          1202118139    NaN               NaN    male  white
9976           444961145    NaN               NaN    male  white
9977           175785523    NaN               NaN  female  black
9978            85633989    NaN               NaN  female    nan
9979           991541906    NaN               NaN    male  white
9980           116010668    NaN               NaN     nan    nan
9981           376927287    NaN               NaN     nan    nan
9982          1246454958    NaN               NaN  female  black
9983            33010187    NaN               NaN     nan    nan
9984          1356310682    NaN               NaN     nan    nan
9985            74059393    NaN               NaN  female  white
9986          3313640937    NaN               NaN     nan    nan
9987  708343452061339648    NaN               NaN  female  black
9988  832653196040380417    NaN               NaN     nan    nan
9989          4182840455    NaN               NaN     nan    nan
9990           569032431    NaN               NaN     nan    nan
9991           580222590    NaN               NaN     nan    nan
9992            38630618    NaN               NaN     nan    nan
9993          1354863247    NaN               NaN     nan    nan
9994  798648351986159616    NaN               NaN    male  white
9995            37661915    NaN               NaN    male  white
9996            20204790    NaN               NaN  female  white
9997            20513486    NaN               NaN     nan    nan
9998  856837313086750720    NaN               NaN     nan    nan
9999  969164460274352128    NaN               NaN  fema

In [7]:
extract = Extractor()
# Add the frequencies of tweets by User
for enum,user in enumerate (lon_users_df['user_id']):
    #print(enum)
    tweet_emoji_count=0
    emojis_count_create_df['Tweets'][enum] = lon_join_df.loc[lon_join_df['user_id'] == user, 'text'].count()
    for tweet in lon_join_df.loc[lon_join_df['user_id'] == user, 'text']:
        if(extract.detect_emoji(tweet)==True):
            tweet_emoji_count=tweet_emoji_count+1
    emojis_count_create_df['Tweets_with_Emoji'][enum] = tweet_emoji_count

In [8]:
# Store table
emojis_count_create_df.to_csv("/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/london_per_user_tweet_counts.csv",encoding='utf-8', index=False)